# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="Toria_workspace")
exp = Experiment(workspace=ws, name="Toria_workspace")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: Toria_workspace
Azure region: eastus
Subscription id: 784d8740-4c70-4d20-8f45-b2bfecf94029
Resource group: udacityprojects


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
compute_target_name = "vacompute"
compute_cluster = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_V2", max_nodes = 4)
my_target =  ComputeTarget.create(ws, compute_target_name, compute_cluster)
### YOUR CODE HERE ###

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [31]:
ws = Workspace.from_config()
experiment_name = 'Personalloans_hyperdrive'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [34]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal,choice
import os
import shutil
from azureml.core import ScriptRunConfig
from azureml.core import Environment


# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.4)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "learning_rate": normal(10, 3),
    "keep_probability": uniform(0.05, 0.1),
    "--C": uniform(0.02, 1),
    "--max_iter": choice(50, 100, 200),

}) 

if "training" not in os.listdir():
    os.mkdir("training")
    print("Path created")
    shutil.copy("trains.py","./training")
    print('copied')

#TODO: Create your estimator and hyperdrive config
compute_target = ws.compute_targets['vacompute']
src = ScriptRunConfig(source_directory='.',
                      script='trains.py',
                      arguments=['--kernel', 'linear', '--penalty', 1.0],
                      compute_target=compute_target,
                      environment = Environment.get(workspace=ws, name='AzureML-Tutorial'))

hyperdrive_run_config = HyperDriveConfig(estimator=src,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=20,
                             max_concurrent_runs=4)

In [35]:
#TODO: Submit your experiment
from azureml.core import Experiment

run = Experiment(ws,'Personalloans_hyperdrive').submit(src)
run.wait_for_completion(show_output=True)

RunId: Personalloans_hyperdrive_1610931902_25c40c57
Web View: https://ml.azure.com/experiments/Personalloans_hyperdrive/runs/Personalloans_hyperdrive_1610931902_25c40c57?wsid=/subscriptions/784d8740-4c70-4d20-8f45-b2bfecf94029/resourcegroups/udacityprojects/workspaces/Toria_workspace

Streaming azureml-logs/55_azureml-execution-tvmps_4f1d66d04d6907de7c68933941e204715d30e2b73fdac290037e0d0395ba4afd_d.txt

2021-01-18T01:05:27Z Starting output-watcher...
2021-01-18T01:05:27Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
2021-01-18T01:05:28Z Executing 'Copy ACR Details file' on 10.0.0.5
2021-01-18T01:05:28Z Copy ACR Details file succeeded on 10.0.0.5. Output: 
>>>   
>>>   
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_d97f122bdcaea8212b2eadb5cef56a6d
2c11b7cecaa5: Pulling fs layer
04637fa56252: Pulling fs layer
d6e6af23a0f3: Pulling fs layer
b4a424de92ad: Pulling fs layer
3e5d9ee64909: Pulling fs layer
3a846111ff22: Pulling fs layer
93a5020

{'runId': 'Personalloans_hyperdrive_1610931902_25c40c57',
 'target': 'vacompute',
 'status': 'Completed',
 'startTimeUtc': '2021-01-18T01:05:26.197128Z',
 'endTimeUtc': '2021-01-18T01:08:44.21204Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'e63e9333-2bdc-4518-acde-41dd2ed3eed0',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'trains.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--kernel', 'linear', '--penalty', '1'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'vacompute',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'priority': None,
  'credentialPassthrough': False,
  'environment': {'name': 'AzureML-Tutorial',
   'version': '60',
   'python'

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [38]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service